# Writing Web Crawlers

- 前面的兩章，我們檢視的是人造的罐頭例子。
- 在這一章，我們要來看真實世界中的例子：
    1. 爬取多重頁數
    1. 甚至，爬取多重網址
- 爬取資料切記：你佔用了多少頻寬、盡最大的努力讓伺服器不要負擔太重

## Traveling a Single Domain

- 任務：找到Eric Idle在Wiki上的網頁，找尋數目最少的、把你帶至Kevin Bacon Wiki網頁的連結 (six-degree Wikipedia solution finder)。
- 減輕Wiki伺服器負責的方式：應該考慮Wiki API (https://www.mediawiki.org/wiki/API:Main_page)

- 抓取單一的wiki網並不難：

In [3]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen('http://en.wikipedia.org/wiki/Kevin_Bacon')
bs = BeautifulSoup(html, 'html.parser')

for link in bs.find_all('a'):                            # html 標記 a 標示：超連結(hyperlinke)
    if 'href' in link.attrs:                             # a 最重要的屬性是：href，標示要連結的網址
        print(link.attrs['href'])

/wiki/Wikipedia:Protection_policy#semi
#mw-head
#p-search
/wiki/Kevin_Bacon_(disambiguation)
/wiki/File:Kevin_Bacon_SDCC_2014.jpg
/wiki/Philadelphia
/wiki/Pennsylvania
/wiki/Kyra_Sedgwick
/wiki/Sosie_Bacon
#cite_note-1
/wiki/Edmund_Bacon_(architect)
/wiki/Michael_Bacon_(musician)
/wiki/Holly_Near
http://baconbros.com/
#cite_note-2
#cite_note-actor-3
/wiki/Footloose_(1984_film)
/wiki/JFK_(film)
/wiki/A_Few_Good_Men
/wiki/Apollo_13_(film)
/wiki/Mystic_River_(film)
/wiki/Sleepers
/wiki/The_Woodsman_(2004_film)
/wiki/Fox_Broadcasting_Company
/wiki/The_Following
/wiki/HBO
/wiki/Taking_Chance
/wiki/Golden_Globe_Award
/wiki/Screen_Actors_Guild_Award
/wiki/Primetime_Emmy_Award
/wiki/The_Guardian
/wiki/Academy_Award
#cite_note-4
/wiki/Hollywood_Walk_of_Fame
#cite_note-5
/wiki/Social_networks
/wiki/Six_Degrees_of_Kevin_Bacon
/wiki/SixDegrees.org
#cite_note-walk-6
#Early_life_and_education
#Acting_career
#Early_work
#1980s
#1990s
#2000s
#2010s
#Advertising_work
#Personal_life
#Six_Degrees_of_Kevi

- 上面的碼，你可以找到Philadelphia, Pennsylvania, Apollo_13_等相關的連結。
- 但是也有很多是你不想要的，如，/wiki/Wikipedia:General_disclaimer、//en.wikipedia.org/wiki/Wikipedia:Contact_us 等。
- 這些是表頭、表尾、側欄等每個wiki網頁都有的訊息。
- 如果你仔細檢視指向其他Wiki文章網頁的連結，你會發現這些連結有三個共通處：
    1. 均位於 div 之中，而且 id 都設為 bodyContent <br>
        a. div 定義了HTML檔案中的一個分段，用來儲存格式化的HTML碼或執行某些JavaScript的動作
    1. URL 中不含冒號
    1. URL 以 /wiki/開始。
- 以上三個條件，可以以下的正則表達式來代表：<img src = 'chap3_re.png'><br> ^表示是字串開始；(...)完全符合圓括弧中的正則表達式；(?!...)表示沒有...所代表的字串；. 表示除了換行外的任何字元；*表示某(些)字元出現零到無限多次；$表示字串的最末端。

In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

html = urlopen('http://en.wikipedia.org/wiki/Kevin_Bacon')
bs = BeautifulSoup(html, 'html.parser')
for link in bs.find('div', {'id':'bodyContent'}).find_all(
    'a', href=re.compile('^(/wiki/)((?!:).)*$')):
    if 'href' in link.attrs:
        print(len(link.attrs['href']))


34
18
18
19
17
30
30
16
27
16
20
22
25
14
30
18
18
20
25
33
16
20
24
20
16
24
23
25
30
19
9
19
24
31
26
29
85
18
19
28
21
32
20
18
30
36
51
25
21
26
17
29
33
18
25
19
33
21
10
26
16
17
15
15
16
20
18
22
26
19
14
18
27
16
27
19
18
26
29
27
33
20
21
16
23
30
24
18
16
27
19
24
20
18
32
33
22
21
33
18
22
12
36
24
20
17
19
20
16
17
22
30
26
17
10
29
15
17
20
25
15
30
15
19
20
18
111
20
29
24
14
20
19
23
28
42
19
47
21
20
37
20
17
19
9
20
15
13
32
30
26
17
21
15
18
15
15
39
18
18
19
24
23
32
12
16
31
19
20
18
29
21
23
21
18
23
37
24
30
17
28
28
26
27
47
40
41
22
25
38
38
16
36
56
25
16
16
27
20
20
18
23
30
36
50
32
39
39
30
27
27
22
25
24
75
20
79
19
85
29
31
49
30
25
29
38
16
19
19
33
33
19
33
19
26
82
19
22
61
30
76
19
18
47
19
47
19
19
19
31
96
32
89
22
89
25
89
24
111
19
24
48
17
48
20
20
19
29
69
26
22
10
32
21
14
29
18
26
32
37
20
50
19
20
18
19
19
19
22
20
15
16
28
21
22
15
18
17
20
22
25
20
23
19
17
20
21
79
19
21
25
16
20
17
17
19
17
19
18
18
19
18
16
17
18
17
19
17
19
18
19
15
19
2

- 上面的動作，雖然有趣，但用處不大。
- 我們需要做的是，把上面的程式碼轉換成類似下面的樣子：
    1. 單一函式 getLinks 讀入一個URL為 /wiki/<Article_Name>格式的 wiki 文章，回傳一個表列的所有連結到的、格式相同的文章URL。    
    1. 一個主要函式，提供一個起啟文章以之呼叫getLinks，從回傳之表列中，隨機選取一個文章連結，再度呼叫getLink，一直到你停止程式，或是新找到的網頁沒有文章連結為止。
- 下面的程式示範了如何從一個網頁進入該網頁中的某個連結的網頁。不過，在網頁間遊走，還未能解決六度wikw的問題。我們還需要儲存、分析找到的資料。我們在第六章繼續這個問題。

In [3]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import datetime
import random
import re

random.seed(datetime.datetime.now())

def getLinks(articleUrl):
    html = urlopen('http://en.wikipedia.org{}'.format(articleUrl))
    bs = BeautifulSoup(html, 'html.parser')
    return bs.find('div', {'id':'bodyContent'}).find_all('a',
                                                        href = re.compile('^(/wiki/)((?!:).)*$'))

links = getLinks("/wiki/Kevin_Bacon")

while len(links) > 0:
    newArticle = links[random.randint(0, len(links)-1)].attrs['href']
    print(newArticle)
    links = getLinks(newArticle)
                                                         

/wiki/JFK_(film)
/wiki/Warren_Commission
/wiki/John_Hart_Ely
/wiki/Ninth_Amendment_to_the_United_States_Constitution
/wiki/Third_Amendment_to_the_United_States_Constitution
/wiki/Convention_to_propose_amendments_to_the_United_States_Constitution
/wiki/New_York_Circular_Letter
/wiki/War_Powers_Clause
/wiki/Concurrent_powers
/wiki/Vesting_Clauses
/wiki/Impeachment_in_the_United_States
/wiki/Divided_government_in_the_United_States
/wiki/List_of_United_States_Representatives_who_switched_parties
/wiki/Closed_sessions_of_the_United_States_House_of_Representatives


KeyboardInterrupt: 

## Crawling an Entire Site

- 上面的程式是隨機選取某網頁中的一個連結，進入該連結的網頁。
- 如果我們需要有系統的某個網站中的每個網頁呢？
- 爬取整個網站，是一件十分耗費記憶體的工作。最適合的解決之道是使用已有的資料庫(database)來儲存取得的資料。第六章介紹兩種方式CSV、MySQL。
- 爬取整個網站有其有用之處：
    1. 產生網頁地圖：
    1. 搜集資料：
- 一般爬取整個網頁的方式是：
    1. 從最上層 (比如，主網頁) 開始，找到該頁上的所有連結。
    1. 每個找到的連結都爬過一遍。
    1. 第二步找到的連結的網頁中，再把所有連結找出來，都爬一遍。
    1. 做到沒有連結為止。
- 這樣做子，很快地，要找的網頁數量就會變得非常大了。比如：每頁有10個連結，而該網站有五頁那麼多，則我們要找的網頁數量為10的5次方，100,000。
- 要減少要爬的網頁數量，很重要的一點是避免重複爬取相同網頁。方法是：把找到的內部連結格式一致化，並且放在集合之中，以供查詢。


In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

pages = set()
def getLinks(pageUrl):
    global pages
    html = urlopen('http://en.wikipedia.org%s' %pageUrl)
    bs = BeautifulSoup(html, 'html.parser')
    for link in bs.find_all('a', href=re.compile('^(/wiki/)')):   # 未使用上面完整的正則表達式，
        if 'href' in link.attrs:
            if link.attrs['href'] not in pages:
                newPage = link.attrs['href']
                print(newPage)
                pages.add(newPage)
                getLinks(newPage)         # 遞迴(recursive)
getLinks('')  # 因為在urlopen()中，pageUrl是附加在wiki主網頁上的，故傳入''等於從主網頁開始爬

# 因為Python有個內建的遞迴(recursive)次數的限制：1000
# 因為wiki網頁包括的連結太多，會到到上數限制，故這個程式會自動crash
# 不過，對沒有包含那麼多連結的網頁，上述程式適用良好。

## Collecting Dtat Across an Entire Site

- 我們來試試一個可以搜集標題、內容的第一段、編輯該頁的連結(如果有)的爬蟲。
- 先看看網站上的幾個網頁，來看看要收集的資料的格式、位置。 wiki網站中，
    1. 標題都在h1 -> span 標記中，而且這是唯一的h1標記。
    1. 所有的正文都在 div#bodyContent 標記下。但如果只想第一段，應使用 div#mw-context-text -> p (只選擇第一段標記)。除了某些檔案網頁，沒有內文。
    1. 編輯這個連結，只出現在文章網頁。如果有，可以在：li#ca-edit 標記中找到，位於 li#ca-edit -> span -> a。
    1. 修改一下之前的程式：

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

pages =  set()

def getLinks(pageUrl):
    global pages
    html = urlopen('http://en.wikipedia.org%s' % pageUrl)
    bs = BeautifulSoup(html, 'html.parser')
    try:
        print(bs.h1.get_text())          # 這個順序是依照出現機率高低排列
        print(bs.find(id='mw-content-text').find_all('p')[0])
        print(bs.find(id='ca-edit').find('span').find('a'.attrs['href'])) 
    except AttributeError:               # 這樣子的做法，缺點是：不知道缺失的是哪一個項目
        print('This page is missing something! Continuing.')
        
    for link in bs.find_all('a', href=re.compile('^(/wiki/)')):
        if 'href' in link.attrs:
            if link.attrs['href'] not in pages:
                newPage = link.attrs['href']
                print('-'*20)
                print(newPage)
                pages.add(newPage)
                getLinks(newPage)
                
getLinks('')

- 處理redirect: r = requests.get('http://github.com', allow_redirects = True)
- 上面的程式只是把擷取的內容印出來，在第五章會講資料的儲存

## Crawling across the internet

- 之前的程式，我們只爬取內部連結，也就是通往同一網站中其他網頁的連結。
- 這裏，我們開始爬取外部連結。
- 爬取外部連結要特別注意，因為你可能在不知不覺中就跑到內容不恰當或是任何不當的網站。
- 在開始爬取外部連結時，應特別注意：
    1. 你想得到什麼資料？能不能只爬取一些事先定義好的網站就得到這些資料？
    1. 到達某個特定網站時，是要遇到第一個外部連結就出去呢，還是在同一個網站留久一點？
    1. 有沒有什麼情況是你不想爬這個網頁的？非中文網站要爬嗎？
    1. 如何保護自己不會違法？(第十八章談這個議題)

In [3]:
from urllib.request import urlopen
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import re
import datetime
import random

pages = ()
random.seed(datetime.datetime.now())

# Retrieves a list of all internal links found on a page

def getInternalLinks(bs, includeUrl):
    includeUrl = '{}://{}'.format(urlparse(includeUrl).scheme, urlparse(includeUrl).netloc)
    internalLinks = []
    # Finds all links that begin with a "/"
    for link in bs.find_all('a',
                           href=re.compile('^(/|.*'+ includeUrl + ')')):
        if link.attrs['href'] is not None:
            if link.attrs['href'] not in internalLinks:
                if (link.attrs['href'].startswith('/')):
                    internalLinks.append(includeUrl+link.attrs['href'])
                else:
                    internalLinks.append(link.attrs['href'])
    return internalLinks

# Retrieves a list of all externallinks found on a apge

def getExternalLinks(bs, excludeUrl):
    externalLinks = []
    for link in bs.find_all('a',
                           href=re.compile('^(http|www)((?!'+excludeUrl+').)*$')):
        if link.attrs['href'] is not None:
            if link.attrs['href'] not in externalLinks:
                externalLinks.append(link.attrs['href'])
    return externalLinks

def getRandomExternalLink(startingPage):
    html = urlopen(startingPage)
    bs = BeautifulSoup(html, 'html.parser')
    externalLinks = getExternalLinks(bs,
                                    urlparse(startingPage).netloc)
    if len(externalLinks) == 0:
        print('No external links, looing around the site for one')
        domain = '{}://{}'.format(urlparse(startingPage).scheme, urlparse(startingPage).netloc)
        internalLinks = getInternalLinks(bs, domain)
        return getRandomExternalLink(internalLinks[random.randint(0, len(internalLinks)-1)])
    else:
        return externalLinks[random.randint(0, len(externalLinks)-1)]
    
def followExternalOnly(startingSite):
    externalLink = getRandomExternalLink(startingSite)
    print('Random external link is: %s' % externalLink)
    followExternalOnly(externalLink)
    
followExternalOnly('http://oreilly.com')    

Random external link is: https://twitter.com/oreillymedia
No external links, looing around the site for one


ValueError: empty range for randrange() (0,0, 0)

- 上面程式的流程圖
<img src ='figure3-1.png'>

### urllib.parse

- urllib.parse 這個模組提供了具以下功能的函式：
    1. 把 Uniform Resource Locator (URL)字串分解為其成份
    1. 把URL的成份合併成為完整URL
    1. 基於一個base URL，將相對URL轉為絕對URL
- urllib.parse.urlparse(urlstring)把一個URL剖析成六個成份，回傳為一個有六個成員的named tuple。分別對應於URL的一般結構：<br>
  scheme ://netloc/path; parameters?query#fragment. 
- urlparse()回傳的值是一個name tuple，故可以用index或名字來取得其值：<br>
    Attribute&nbsp;&nbsp;&nbsp;&nbsp;Index <br>
    scheme&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;0 <br>
    netloc&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;1 <br>
    path&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;2 <br>
    params&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;3 <br>
    query&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;4 <br>
    fragment&nbsp;&nbsp;&nbsp;&nbsp;5 <br>
    username <br>
    password <br>
    hostname <br>
    port

In [5]:
# 如果只要爬取外部連結，可以在上面的程式下，加入下面的函式：

allExtLinks = set()
allIntLinks = set()

def getAllExternalLinks(siteUrl):
    html = urlopen(siteUrl)
    domain = '%s://%s' %(urlparse(siteUrl).scheme,
                         urlparse(siteUrl).netloc)
    bs = BeautifulSoup(html, 'html.parser')
    internalLinks = getInternalLinks(bs, domain)
    externalLinks = getExternalLinks(bs, domain)
    
    print('-'*4,'External','-'*40)
    for link in externalLinks:
        if link not in allExtLinks:
            allExtLinks.add(link)
            print(link)
            
    print('-'*4,'Internal','-'*40)
    for link in internalLinks:
        if link not in allIntLinks:
            allIntLinks.add(link)
            getAllExternalLinks(link)

allIntLinks.add('http://oreilly.com')
getAllExternalLinks('http://oreilly.com')
    
    


---- External ----------------------------------------
https://www.oreilly.com
https://www.oreilly.com/online-learning/enterprise.html
https://learning.oreilly.com/accounts/login-check/
https://www.oreilly.com/online-learning/try-now.html
https://www.oreilly.com/online-learning/index.html
https://www.oreilly.com/online-learning/individuals.html
https://www.oreilly.com/online-learning/teams.html
https://www.oreilly.com/online-learning/government.html
https://www.oreilly.com/online-learning/academic.html
https://www.oreilly.com/online-learning/features.html
https://www.oreilly.com/online-learning/custom-services.html
https://www.oreilly.com/online-learning/pricing.html
https://www.oreilly.com/whats-new.html
https://www.oreilly.com/online-learning/certifications.html
https://www.oreilly.com/library/view/cissp-second-edition/9780135180488/
https://www.oreilly.com/library/view/exam-ref-az-900/9780135732199/
https://www.oreilly.com/library/view/aws-certified-solutions/9780135229415/
https://

HTTPError: HTTP Error 404: Not Found

- 上述程式流程圖
<img src = "figure3-2.png">